In [ ]:
#Code to get the load_csv_database function.
#We pass this function as context so there's no need to include it when we parse the notebook
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))
from spider2_utils import load_csv_database

-setup-

In [ ]:
_database = load_csv_database("E_commerce", rows_limit=-1)
import pandas as pd

customers = _database["customers"]
orders = _database["orders"]
order_items = _database["order_items"]
products = _database["products"]
geolocation = _database["geolocation"]
leads_closed = _database["leads_closed"]
leads_qualified = _database["leads_qualified"]
order_payments = _database["order_payments"]
order_reviews = _database["order_reviews"]
products = _database["products"]
product_category_name_translation = _database["product_category_name_translation"]
sellers = _database["sellers"]

# Question

Could you tell me the number of orders, average payment per order and customer lifespan in weeks of the 3 custumers with the highest average payment per order, where the lifespan is calculated by subtracting the earliest purchase date from the latest purchase date in days, dividing by seven, and if the result is less than seven days, setting it to 1.0?

# Step 1: Aggregate Customer Data
Calculate the total number of orders, total payment, and the time range of orders for each customer.

In [ ]:

# Assuming customers, orders, and order_payments DataFrames are already loaded
customer_data = orders.merge(order_payments, on='order_id')
customer_data = customer_data.merge(customers, on='customer_id')
date_fmt = "%Y-%m-%d %H:%M:%S"
customer_data = customer_data.groupby('customer_unique_id').agg(
    order_count=('order_id', 'nunique'),
    total_payment=('payment_value', 'sum'),
    first_order_day=('order_purchase_timestamp', lambda x: pd.to_datetime(x, format=date_fmt).min().to_julian_date()),
    last_order_day=('order_purchase_timestamp', lambda x: pd.to_datetime(x, format=date_fmt).max().to_julian_date())
).reset_index()

# Step 2: Calculate Purchase Frequency (PF)
Determine the purchase frequency for each customer.

In [ ]:
customer_data['PF'] = customer_data['order_count']

# Step 3: Calculate Average Order Value (AOV)
Compute the average order value for each customer.

In [ ]:
customer_data['AOV'] = (customer_data['total_payment'] / customer_data['order_count']).round(2)

# Step 4: Calculate Average Customer Length (ACL)
Calculate the time range of orders in weeks for each customer.

In [ ]:
customer_data['ACL'] = customer_data.apply(
    lambda row: 1 if (row['last_order_day'] - row['first_order_day']) < 7 else (row['last_order_day'] - row['first_order_day']) / 7,
    axis=1
)

# Step 5: Select the top 3 customers with highest AOV

In [ ]:
# Select the top 3 customers with highest AOV
top3_customers = customer_data.sort_values(by='AOV', ascending=False).head(3)
top3_customers[['customer_unique_id', 'order_count', 'AOV', 'ACL']]


In [ ]:
# spider2 gold exec results
# customer_unique_id	PF	AOV	ACL
# 0a0a92112bd4c708ca5fde585afaa872	1	13664.08	1
# 763c8b1c9c68a0229c42c9fc6f662b93	1	7274.88	1
# dc4802a71eae9be1dd28f5d788ceb526	1	6929.31	1Í